## 数据探索与可视化

### 实验目标
- 学习并掌握如何使用Python进行数据探索与可视化。
- 理解并应用数据探索的基本方法，包括数据清洗、格式化和描述性统计分析。
- 使用Matplotlib和Seaborn等可视化工具，绘制不同类型的图表，分析数据分布及特征之间的关系。
- 利用pandas_profiling生成数据的交互式报告，全面了解数据的分布、缺失值和异常值。

### 实验要求

1. 使用pandas读取数据集github_bot_processed_data.csv。探索pandas的显示选项，以便查看更多数据行和列。
2. 使用head()方法查看数据的前几行，并使用info()方法查看每列的数据类型，观察两者的区别。
使用describe()方法生成数据的描述性统计信息。
3. 对数据进行格式化处理，并展示不同格式（例如，日期、货币、百分比等）的效果。
4. 对public_repos、public_gists、followers、following等列进行对数变换，并查看其影响。
5. 使用Matplotlib绘制图表
    - 绘制条形图：展示label列的类别分布。
    - 绘制堆积柱状图：展示多个布尔特征（如site_admin、company等）的分布。
    - 绘制直方图：展示log_public_repos的对数变换后的数据分布。
    - 绘制散点图：展示public_repos与followers之间的关系。
    - 绘制散点矩阵：展示多个数值型特征之间的成对关系。
6. 使用Seaborn绘制图表
    - 绘制箱线图：展示不同label类别下log_followers的分布。
    - 绘制成对图：展示不同特征之间的成对关系，并根据label分类。
    - 绘制热图：展示log_public_repos、log_public_gists、log_followers和log_following等特征之间的相关性。
    - 绘制小提琴图：展示label与log_followers之间的分布差异。
7. 使用pandas_profiling.ProfileReport()生成交互式数据分析报告，分析数据的统计分布、缺失值、异常值等。

In [ ]:
import pandas as pd
import numpy as np
import re
import datetime


def turn_timestamp(x):
    pattern = '\d+'
    list = re.findall(pattern,x)
    #print(list)
    t = datetime.datetime(int(list[0]),int(list[1]),int(list[2]),int(list[3]),int(list[4]),int(list[5]))
    return t.timestamp()

def turn_bool(x):
    if x == True:
        return 1
    else:
        return 0


data = pd.read_csv('github_bot_processed_data.csv')
pd.set_option('display.min_rows',100)
pd.set_option('display.max_columns',None)

#print(data.head())   #data的前几行
#print(data.info())    #每列的信息，包括数据类型等
#print(data.describe())  #每列的统计信息，包括极值，平均值，四分位值等

data[['log_public_repos','log_public_gists','log_followers','log_following']] = data[['public_repos','public_gists','followers','following']].map(np.log1p)
data[['created_at','updated_at']] = data[['created_at','updated_at']].map(turn_timestamp)
data[['site_admin','company','blog','location','hireable']] = data[['site_admin','company','blog','location','hireable']].map(turn_bool)

path = 'github_data.csv'
data.to_csv(path)


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_csv('github_data.csv')
count = data['label'].value_counts()
human = data[data['label'] == 'Human']
bot = data[data['label'] == 'Bot']



x = ['Human','Bot']
y = [count.Human,count.Bot]

p1 = plt.bar(x,y,label = True)
plt.title('label')
plt.bar_label(p1,label_type = 'edge')
#plt.show()



labels = ['Human','Bot']

print(human['site_admin'].value_counts())
admin = [sum(human['site_admin']),sum(bot['site_admin'])]
company = [sum(human['company']),sum(bot['company'])]
blog = [sum(human['blog']),sum(bot['blog'])]
location = [sum(human['location']),sum(bot['location'])]
hireable = [sum(human['hireable']),sum(bot['hireable'])]


fig,ax = plt.subplots(figsize=(2,10),dpi=200)
p2 = ax.bar(labels,admin,bottom=None,label='Admin')
p3 = ax.bar(labels,company,bottom=admin,label='Company')
p4 = ax.bar(labels,blog,bottom=company,label='Blog')
p5 = ax.bar(labels,location,bottom=blog,label='Location')
p6 = ax.bar(labels,hireable,bottom=location,label='Hireable')

plt.bar_label(p2,label_type= 'center')
plt.bar_label(p3,label_type= 'center')
plt.bar_label(p4,label_type= 'center')
plt.bar_label(p5,label_type= 'center')
plt.bar_label(p6,label_type= 'center')
plt.title('component')
plt.show()




In [ ]:
import pandas as pd


data = pd.read_csv('github_data.csv')
stats = data['log_public_repos']
plt.hist(stats,bins=50)
plt.xlabel('log_public_repos')
plt.ylabel('frequency')
plt.title('public_repo')
plt.show()

repo = data['public_repos']
follower = data['followers']

upper_repo = 4000
upper_follower = 40000
processed_repo = []
processed_follower = []
length = len(repo)

for i in range(length):
    if repo[i] < upper_repo and follower[i] < upper_follower:
        processed_repo.append(repo[i])
        processed_follower.append(follower[i])
    


plt.scatter(processed_repo,processed_follower)
plt.show()





In [ ]:
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt


data = pd.read_csv('github_data.csv')
sns.boxplot(x='label',y='log_followers',data=data)
plt.title('different labels log_followers')
plt.show()

df = data[['label','log_public_repos','log_public_gists','log_followers','log_following']]
sns.pairplot(df,hue='label')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from pandas_profiling import ProfileReport

df = pd.read_csv('github_data.csv')
data = df.iloc[0:20,:]
df = data[['log_public_repos','log_public_gists','log_followers','log_following']]
sns.heatmap(df,annot=True,fmt=".1f",cmap='viridis',vmax = 8)
plt.yticks(rotation=0)
plt.xticks(rotation=45)
plt.figure(figsize=(6,6),dpi=200)
plt.show()


df = pd.read_csv('github_data.csv')
sns.violinplot(df,x='label',y='log_followers')
plt.show()




 
# 加载数据
df = pd.read_csv('github_data.csv')
 
# 生成报告
profile = ProfileReport(df, title="Your Dataset Profile", explorative=True)
 
# 在 Jupyter Notebook 中显示报告
profile.to_notebook_iframe()
